In [ ]:
%load_ext autoreload

In [ ]:
%autoreload

import h2o
import create_data as create
import pandas as pd
import utilities as utils

from h2o.automl import H2OAutoML

In [ ]:
data = create.CreateTrainingData()
train = data.create_training_data()
test = create.create_test_data()
target = create.create_target_data()

In [ ]:
h2o.init()

In [ ]:
train.head()

In [ ]:
# Fill nans with 0 in training data
train.fillna(0, inplace=True)

X_train = h2o.H2OFrame(train)
y = 'pv_measurement'

X = X_train.columns
X.remove(y)

In [ ]:
autoML = H2OAutoML(max_models=30, seed=1, nfolds=5)
autoML.train(x=X, y=y, training_frame=X_train)

In [ ]:
# Remove features that are not in the training set
test_pd = test[X]

# Set nans to 0 in test_pd
# test_pd.fillna(0, inplace=True)

test = h2o.H2OFrame(test_pd)

prediction = autoML.predict(test)

In [ ]:
preds_df = prediction.as_data_frame()
preds_df[preds_df < 0] = 0

results = pd.concat([test_pd.reset_index(), preds_df], axis=1)
results['date_forecast'] = pd.to_datetime(results['date_forecast'])
results.set_index('date_forecast', inplace=True)
results.rename(columns={'predict': 'pv_prediction'}, inplace=True)

utils.save_to_csv(results, 'h2o_nans_to_0')